**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "annotation_ccre")
fnames = dir(fdiry)
fnames

[1] "peak.annotation.ccre_v3.bed.gz" "peak.annotation.ccre_v4.bed.gz"

In [3]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_ccre")
fnames = dir(fdiry)
fnames

[1] "description.tsv"

## Import data

In [4]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_ccre")
fname = "description.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath)

dat_cnames = dat
print(dim(dat))
dat

Rows: 11 Columns: 2
── Column specification ─────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): Name, Description

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 11  2


Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,Start position
End,End position
Name,Name
Score,Score
Strand,[+-.]; Use '.' if no strand is assigned.
ThickStart,The starting position at which the feature is drawn thickly
ThickEnd,The ending position at which the feature is drawn thickly
ItemRgb,"An RGB value of the form R,G,B (e.g. 255,0,0)"


In [5]:
### init
annotation = "ccre_v4"
cnames = dat_cnames$Name
cnames = c("Chrom_ATAC", "Start_ATAC", "End_ATAC", cnames, "Overlap")

###
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "annotation_ccre")
fname = "peak.annotation.ccre_v4.bed.gz"
fpath = file.path(fdiry, fname)

### read data
dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)
dat = dat %>% dplyr::mutate(
    Peak_ATAC  = paste(Chrom_ATAC, Start_ATAC, End_ATAC, sep="_"),
    Annotation = annotation,
    Label      = Category
)

### assign and show
dat_peak_annot = dat
print(dim(dat))
head(dat)

[1] 480473     18


Chrom_ATAC,Start_ATAC,End_ATAC,Chrom,Start,End,Name,Score,Strand,ThickStart,ThickEnd,ItemRgb,Category,Note,Overlap,Peak_ATAC,Annotation,Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
chr1,10015,10442,chr1,10033,10250,EH38E2776516,0,.,10033,10250,"225,225,225",Low-DNase,All-data/Full-classification,217,chr1_10015_10442,ccre_v4,Low-DNase
chr1,10015,10442,chr1,10385,10713,EH38E2776517,0,.,10385,10713,"225,225,225",Low-DNase,All-data/Full-classification,57,chr1_10015_10442,ccre_v4,Low-DNase
chr1,16015,16477,chr1,16097,16381,EH38E3951272,0,.,16097,16381,"225,225,225",Low-DNase,All-data/Full-classification,284,chr1_16015_16477,ccre_v4,Low-DNase
chr1,17237,17772,chr1,17343,17642,EH38E3951273,0,.,17343,17642,"225,225,225",Low-DNase,All-data/Full-classification,299,chr1_17237_17772,ccre_v4,Low-DNase
chr1,28903,29613,chr1,29320,29517,EH38E3951274,0,.,29320,29517,"225,225,225",Low-DNase,All-data/Full-classification,197,chr1_28903_29613,ccre_v4,Low-DNase
chr1,115411,115986,chr1,115562,115912,EH38E3951280,0,.,115562,115912,"0,176,240",CA-CTCF,All-data/Full-classification,350,chr1_115411_115986,ccre_v4,CA-CTCF


## Arrange and summarize

In [6]:
dat = dat_peak_annot
dat = dat %>% 
    dplyr::group_by(Peak_ATAC, Annotation, Label) %>%
    dplyr::summarise(Count = n(), .groups = "drop")

dat = dat %>% dplyr::rename("Peak" = "Peak_ATAC")

### assign and show
dat_peak_annot_count = dat
print(dim(dat))
head(dat)

[1] 277309      4


Peak,Annotation,Label,Count
<chr>,<chr>,<chr>,<int>
chr10_100009096_100010466,ccre_v4,CA-H3K4me3,2
chr10_100009096_100010466,ccre_v4,Low-DNase,1
chr10_100009096_100010466,ccre_v4,PLS,1
chr10_100009096_100010466,ccre_v4,pELS,1
chr10_100020710_100021134,ccre_v4,Low-DNase,2
chr10_100045961_100046653,ccre_v4,CA-CTCF,1


## Save results

In [7]:
fdiry = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input", "summary")
fname = "peak.summary.ccre_v4.tsv"
fpath = file.path(fdiry, fname)

dat = dat_peak_annot_count
write_tsv(dat, fpath)